In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)
    
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/

print("Available Pretrained Models:\n")
!ls ~/.keras/models

In [ ]:
imageWidth = 224
imageHeight = 224
train_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Training/'
validation_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test/'
train_samples = 31688
validation_samples = 10657
batch_size = 64

In [ ]:
print(os.listdir("../input/fruits/fruits-360_dataset/fruits-360/Test/"))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(imageHeight, imageWidth),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(imageHeight, imageWidth),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
K.set_learning_phase(1)
inceptionModel = applications.ResNet50(weights='imagenet', include_top=False)
fcLayer = inceptionModel.output
fcLayer = GlobalAveragePooling2D()(fcLayer)

fcLayer = Dense(512, activation='relu')(fcLayer)
Predictions = Dense(75, activation='softmax')(fcLayer)
FinalModel = Model(inputs=inceptionModel.input, outputs=Predictions)

In [ ]:
FinalModel.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
history_pretrained = FinalModel.fit_generator(
train_generator,
epochs=3, shuffle = True, verbose = 1, validation_data = validation_generator)

K.set_learning_phase(0)

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [ ]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in FinalModel.outputs])
tf.train.write_graph(frozen_graph, "", "my_model.pb", as_text=False)